**Q4.** Can you detect cascading failures as delays in one airport create delays in other?

In [1]:
#Import libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce 
from functools import reduce

In [ ]:
#Read data
data_2003 = pd.read_csv("2003.csv.bz2")
data_2004 = pd.read_csv("2004.csv.bz2")

In [3]:
#Prepare data and view fields
data_2004 = data_2004.sample(6488540)
datamerged = pd.concat([data_2003, data_2004])
datamerged.isna().sum()

Year                        0
Month                       0
DayofMonth                  0
DayOfWeek                   0
DepTime                217705
CRSDepTime                  0
ArrTime                241625
CRSArrTime                  0
UniqueCarrier               0
FlightNum                   0
TailNum                 26455
ActualElapsedTime      241626
CRSElapsedTime              1
AirTime                241625
ArrDelay               241626
DepDelay               217705
Origin                      0
Dest                        0
Distance                    0
TaxiIn                      0
TaxiOut                     0
Cancelled                   0
CancellationCode     12807743
Diverted                    0
CarrierDelay          2672742
WeatherDelay          2672742
NASDelay              2672742
SecurityDelay         2672742
LateAircraftDelay     2672742
dtype: int64

In [4]:
#Cleanse data and merge
datamerged.dropna(how="all", axis=1, inplace=True)
datamerged = datamerged.drop_duplicates()
datamerged.head()

Year  Month  DayofMonth  DayOfWeek  DepTime  CRSDepTime  ArrTime  \
0  2003      1          29          3   1651.0        1655   1912.0   
1  2003      1          30          4   1654.0        1655   1910.0   
2  2003      1          31          5   1724.0        1655   1936.0   
3  2003      1           1          3   1033.0        1035   1625.0   
4  2003      1           2          4   1053.0        1035   1726.0   

   CRSArrTime UniqueCarrier  FlightNum  ... TaxiIn  TaxiOut  Cancelled  \
0        1913            UA       1017  ...      5       17          0   
1        1913            UA       1017  ...      2       26          0   
2        1913            UA       1017  ...      5       17          0   
3        1634            UA       1018  ...      6       11          0   
4        1634            UA       1018  ...     13       46          0   

   CancellationCode  Diverted  CarrierDelay WeatherDelay NASDelay  \
0               NaN         0           NaN          NaN      NaN   
1               NaN         0           NaN          NaN      NaN   
2               NaN         0           NaN          NaN      NaN   
3               NaN         0           NaN          NaN      NaN   
4               NaN         0           NaN          NaN      NaN   

   SecurityDelay  LateAircraftDelay  
0            NaN                NaN  
1            NaN                NaN  
2            NaN                NaN  
3            NaN                NaN  
4            NaN                NaN  

[5 rows x 29 columns]

In [5]:
#Retrieve fields and rereformat them
datamerged= datamerged[['TailNum','Origin','Dest','DepDelay','ArrDelay','DepTime']]

#create column with boolean delay occurence value
def dummy (row): 
    if row <= 0: return 0
    if row > 0: return 1
datamerged['DepDelay']=datamerged['DepDelay'].apply(dummy) 
datamerged['ArrDelay']=datamerged['ArrDelay'].apply(dummy)

datamerged

TailNum Origin Dest  DepDelay  ArrDelay  DepTime
0        N202UA    ORD  MSY       0.0       0.0   1651.0
1        N311UA    ORD  MSY       0.0       0.0   1654.0
2        N317UA    ORD  MSY       1.0       1.0   1724.0
3        N409UA    OAK  ORD       0.0       0.0   1033.0
4        N496UA    OAK  ORD       1.0       1.0   1053.0
...         ...    ...  ...       ...       ...      ...
2216782  N245AA    SAT  ORD       0.0       0.0   1755.0
2038212  N935DL    DCA  ATL       0.0       1.0   1805.0
2209811  N4XAAA    ORD  MSP       1.0       1.0   2129.0
5498898  N751NW    DLH  MSP       0.0       0.0   1250.0
6072217    N426    PHX  STL       0.0       0.0    900.0

[12977055 rows x 6 columns]

In [6]:
datamerged['TailNum'].value_counts()

0         95723
000000    18411
N308SW     6939
N226SW     5860
N293SW     5843
          ...  
N632US        1
N854NW        1
N627US        1
N906S         1
N856NW        1
Name: TailNum, Length: 5840, dtype: int64

In [7]:
datamergedN226SW = datamerged[datamerged['TailNum'] == 'N226SW'].sort_values(by=['DepTime'])
datamergedN226SW  #explore

TailNum Origin Dest  DepDelay  ArrDelay  DepTime
2871418  N226SW    SFO  CIC       1.0       1.0      3.0
1791771  N226SW    IYK  LAX       0.0       0.0    449.0
1256807  N226SW    IYK  LAX       0.0       0.0    452.0
5549768  N226SW    PSC  SEA       0.0       0.0    454.0
1249870  N226SW    PSP  LAX       0.0       1.0    458.0
...         ...    ...  ...       ...       ...      ...
5064348  N226SW    SFO  ACV       1.0       1.0   2306.0
4959471  N226SW    LAX  MRY       1.0       1.0   2319.0
6736788  N226SW    SFO  RDD       1.0       1.0   2320.0
5612405  N226SW    SFO  ACV       1.0       1.0   2342.0
798350   N226SW    ACV  SMF       1.0       0.0   2400.0

[5860 rows x 6 columns]

In [8]:
#Selecting fields
datamergedN226SW = datamergedN226SW[['TailNum','DepDelay','ArrDelay']] #columns to use

#Counting consecutive delays
datamergedN226SW['dep']=datamergedN226SW['DepDelay']+datamergedN226SW['ArrDelay']+datamergedN226SW['DepDelay'].shift(1)
datamergedN226SW['arr']=datamergedN226SW['ArrDelay']+datamergedN226SW['DepDelay']+datamergedN226SW['ArrDelay'].shift(1)
datamergedN226SW2 = datamergedN226SW[(datamergedN226SW["dep"] == 3.0) | (datamergedN226SW["arr"] == 3.0)]

#Counting occurences
datamergedN226SW2 =datamergedN226SW2.groupby("TailNum").count().reset_index(drop=False)
datamergedN226SW2 = datamergedN226SW2[['TailNum','arr']] #sort
datamergedN226SW2.rename(columns={'arr': 'Cascading Failures'}, inplace=True) #rename
datamergedN226SW2 

/var/folders/3z/k4yvc61j3t5gfwy48mp0218w0000gn/T/ipykernel_98276/3358096987.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datamergedN226SW['dep']=datamergedN226SW['DepDelay']+datamergedN226SW['ArrDelay']+datamergedN226SW['DepDelay'].shift(1)
/var/folders/3z/k4yvc61j3t5gfwy48mp0218w0000gn/T/ipykernel_98276/3358096987.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datamergedN226SW['arr']=datamergedN226SW['ArrDelay']+datamergedN226SW['DepDelay']+datamergedN226SW['ArrDelay'].shift(1)


TailNum  Cascading Failures
0  N226SW                 474

In [10]:
#Outputting the rate of cascading delays of total
N226SW_totalcasc =(474/5860)*100

print("Plane with TailNum N226SW has a cascading delay rate of:", N226SW_totalcasc,"% of total flights")

Plane with TailNum N226SW has a cascading delay rate of: 8.088737201365188 % of total flights
